# Homework 5

Austin Gill

* 11: 3
* 16: 1, 2
* 17: 2, 3, 4
* 18: 1.1
* Text Addition

## Problem 11.3

Write a Python program to navigate a robot using the Wavefront algorithm. Demonstrate on a map with multiple obstacles.

## Problem 16.1

Let $z_1=284$, $z_2=257$, $z_3=295$, be measurements from sensors which have normally distributed errors with standard deviations $\sigma_1=10$, $\sigma_2=20$, $\sigma_3=15$, respectively. What is the best estimate for the measured state?

## Problem 16.2

You are given three distance sensors which all measure the same distance. To determine the accuracy of each you run repeated measurements of an object which you setup so that the sensor will return 2 meters. Running 40 measurements you obtain the data listed below. Next you measure the distance of an object ahead of the robot. Sensor $A$ gives 2.4577696, sensor $B$ gives 1.8967743, and sensor $C$ gives 2.1352561. Combine these readings to make a more accurate estimate of the object distance. Hint: you need to figure out the distributions and the data according to the variances.

|    $A$    |    $B$    |    $C$    |
|:---------:|:---------:|:---------:|
| 2.2411549 | 1.8286673 | 2.3295015 |
| 2.3366108 | 1.9243295 | 2.4867167 |
| 1.9687234 | 1.8972737 | 2.5489412 |
| 2.1240351 | 2.0961534 | 1.9834876 |
| 2.3984044 | 1.7985819 | 2.6153805 |
| 2.3523899 | 1.8377782 | 1.8132444 |
| 2.1074266 | 1.8358201 | 2.5563951 |
| 2.4711542 | 1.8875839 | 2.2031291 |
| 2.1998000 | 1.8116113 | 2.3117542 |
| 2.2710086 | 1.8701890 | 2.3495262 |
| 2.3530473 | 1.7646824 | 2.0109293 |
| 2.4391559 | 1.9499153 | 2.5030771 |
| 2.2066306 | 1.9243432 | 2.3561112 |
| 2.3000099 | 1.8309696 | 2.3097754 |
| 2.2235766 | 1.8453219 | 2.2940692 |
| 2.1396901 | 1.8390955 | 2.1904604 |
| 2.0929719 | 1.7978329 | 2.5693897 |
| 2.3154159 | 1.8217245 | 1.9332188 |
| 2.3716302 | 1.9558670 | 2.3002433 |
| 2.2611420 | 1.8654487 | 2.5508342 |
| 2.1415088 | 1.7836290 | 2.6884786 |
| 2.2088487 | 1.9245743 | 2.5037028 |
| 2.2714614 | 1.8918415 | 2.7112663 |
| 2.3345816 | 1.8275421 | 2.1656644 |
| 2.3052296 | 1.8494488 | 2.1940472 |
| 2.1600041 | 1.7632971 | 2.2703708 |
| 2.0630943 | 1.8396972 | 2.6488544 |
| 2.0997821 | 1.8412331 | 2.1828831 |
| 2.3037175 | 1.7761007 | 2.2959535 |
| 2.4536524 | 1.8542271 | 2.0446945 |
| 2.3909478 | 1.8649815 | 2.7852822 |
| 2.1195966 | 1.9533324 | 2.5700007 |
| 2.0205112 | 1.8857815 | 2.1113650 |
| 2.1708006 | 1.7115595 | 2.1215336 |
| 2.0800748 | 1.9403332 | 2.3126032 |
| 2.3332722 | 1.8530670 | 2.4687277 |
| 2.0826115 | 1.8279041 | 2.6104026 |
| 2.2652480 | 1.9058054 | 2.3165716 |
| 2.3734464 | 1.9632258 | 2.0907554 |
| 2.0563260 | 1.9367908 | 2.2130578 |

## Problem 17.2

Assume that one has three different measurements for the location of some object. The three measurements with the covariances are

$$\begin{align}
    (10.5, 18.2)&, \quad\begin{pmatrix}
        0.1 & 0.01 \\
        0.01 & 0.15
    \end{pmatrix} \\
    (10.75, 18.0)&, \quad\begin{pmatrix}
        0.05 & 0.005 \\
        0.005 & 0.05
    \end{pmatrix} \\
    (9.9, 19.1)&, \quad\begin{pmatrix}
        0.2 & 0.05 \\
        0.05 & 0.25
    \end{pmatrix}
\end{align}$$
Fuse this data into one measurement and provide an estimate of the covariance.

## Problem 17.3

Run a simulation on

$$\begin{align*}
    \dot x &= y \\
    \dot y &= -\cos x + 0.5 \sin t
\end{align*}$$

adding noise to the $x$ and $y$ components (with variance = $0.2$ on each). Let $\Delta t=0.1$. Assume that you can observe the first variable, $x$, with variance $0.25$. Record the observations. Write a program to run the EKF on the observed data and compare the state estimate to the original values.

## Problem 17.4

Differential Drive - EKF. The motion equations for a differential drive robot are given below. Assume that the wheels are $5$cm in radius and the wheelbase is $12$cm.

The covariance of the state transition is $V$. Assume that you have a local GPS system that gives $(x,y)$ data subject to Gaussian noise with covariance $W$. The units on the noise are given in cm. If you want to use meters then you will need to divide your noise by 100.

1. Starting at $t=0$, $x=0$, $y=0$, $\theta=0$, predict location when wheel velocities are:
   
   $$\begin{align*}
     0  \leq t \leq 5 &,\quad  \omega_1 =  \omega_2 = 3 \\
     5  \leq t \leq 6 &,\quad  \omega_1 = -\omega_2 = 1 \\
     6  \leq t \leq 10&,\quad  \omega_1 =  \omega_2 = 3 \\
     10 \leq t \leq 11&,\quad -\omega_1 =  \omega_2 = 1 \\
     11 \leq t \leq 16&,\quad  \omega_1 =  \omega_2 = 3 \\
   \end{align*}$$
   assuming that you have Gaussian noise on the omegas per time step. Select $\Delta t=0.2$ (time increment). Test with a standard deviation of $0.1$ on both wheels.
2. Write out the formulas for the Extended Kalman Filter.
3. Apply an Extended Kalman filter to the motion simulation above to track the location of the vehicle. Observations can be simulated by using previous simulation data as actual data, i.e. use this as the observed data ($z_k$). Parameters:
   $$x_{0|0} = (0,0,0)$$
   $$V = \begin{pmatrix}.05 &  .02 & 0.01\\.02& .05& 0.01\\ 0.01& 0.01& .1\end{pmatrix}$$
   $$W = \begin{pmatrix} .08& .02 \\.02&  .07\end{pmatrix}$$
   $$P_{0|0} = \begin{pmatrix}2 &0& 0\\0 &1& 0\\0& 0& 0.5\end{pmatrix}$$
4. Output the $x, y$ locations on a 0.5 sec grid and compare in a plot.
   
   ??
5. The covariance matrix $P$ gives the uncertainly ellipse for the location of the robot. Plot 5 ellipses along the path. This ellipse has major and minor axes given by the eigenvectors of $P$ and the axes lengths are given by the associated eigenvalues.

## Problem 18.1.1 ??

Let the domain be the rectangle $0 \leq x \leq 15$ and $0 \leq y \leq 10$. Place the start position at $(0,5)$. Place the end position at $(15,5)$. Assume you have circular obstacles centered at $(6,4)$ with radius $2$ and at $(8,6)$ with radius $3$. Find a potential function which can navigate the robot from the start to the end position.

1. Plot the resulting path in Python with obstacles included in the map.
2. Compare to the Wavefront approach.